In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

TRAIN_TARGET = "../future-temparture-prediction/Temperature_Train_Target.dat.tsv"

In [2]:
%run ../script/prepocessing.py
Train = TemperatureData('train')
y = np.loadtxt(TRAIN_TARGET)

In [3]:
tempers = Train.temperWVecMean()

In [4]:
tempers.shape

(1800, 11)

年の切れ目359, 719, 1079, 1439, 1799を無視する。

In [5]:
edges = [359, 719, 1079, 1439, 1799]

In [6]:
[len(Train.TemperPlaceId[i]) for i in xrange(11)]

[164, 164, 164, 164, 164, 162, 164, 164, 164, 163, 163]

In [7]:
tempers_place = dict()
learnig_id = dict()
drop_list = list()
for i in xrange(11):
    drop_flag = False
    for edge in edges:
        if edge in Train.TemperPlaceId[i]:
            a = list(Train.TemperPlaceId[i])
            index = a.index([edge])
            drop_list.append([i,index])
            ls = list(Train.TemperPlaceId[i])
            ls.pop(index)
            tempers_place[i] = tempers[ls]
            learnig_id[i] = ls
            drop_flag = True
            break
    if not drop_flag:
        ls = list(Train.TemperPlaceId[i])
        tempers_place[i] = tempers[ls]
        learnig_id[i] = ls

In [8]:
[len(tempers_place[i]) for i in xrange(11)]

[163, 163, 164, 164, 163, 162, 164, 163, 163, 163, 163]

In [9]:
drop_list

[[0, 131], [1, 98], [4, 65], [7, 32], [8, 163]]

In [10]:
for i in xrange(len(drop_list)):
    print Train.TemperPlaceId[drop_list[i][0]][drop_list[i][1]]

1439
1079
719
359
1799


each_place learning

In [11]:
supervisers = [y[learnig_id[i]] for i in xrange(11)]

In [12]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn import grid_search
parameters = {'alpha':[float(i)/100 for i in xrange(0,101)]}
clf = [grid_search.GridSearchCV(Ridge(), parameters, cv=10, scoring='mean_squared_error') for i in xrange(11)]

In [13]:
for i in xrange(11):
    print i
    clf[i].fit(tempers_place[i], supervisers[i])
    print clf[i].predict(np.identity(11))

0
[ 0.73925205  0.00831007  0.07760923 -0.11496033 -0.14845264  0.14958621
  0.10606897  0.08888672 -0.01108878  0.02491205  0.02470168]
1
[-0.02625402  0.81890899  0.14664959 -0.01119781 -0.01061256  0.01337498
  0.0416254  -0.05722061  0.01678903  0.07483866  0.02078398]
2
[ 0.05192627  0.08509273  0.80875213 -0.01029769 -0.03275557  0.06163761
  0.01784571 -0.09905704  0.06412852  0.06199866 -0.01105413]
3
[ 0.17358115 -0.03009986  0.03101626  0.81043239  0.0643825  -0.01211051
  0.06929736 -0.1113946  -0.05138908 -0.0458437  -0.01388901]
4
[-0.03544525 -0.02046994 -0.02838198  0.02536318  0.75139797  0.29822392
 -0.00171048  0.02204578  0.00431965  0.02310748  0.01052007]
5
[-0.14809534 -0.14381326  0.1456384   0.05630641  0.1263742   0.94660658
 -0.05015703  0.00404385  0.00886697  0.14951101 -0.00694425]
6
[ 0.00647589 -0.0544007   0.11597037  0.05441384 -0.02948761  0.10344106
  0.67032077  0.17864986  0.09809684  0.06648278 -0.07664391]
7
[ 0.11152984 -0.05489807 -0.03597372 -0

In [14]:
for i in xrange(11):
    print i, ":", clf[i].best_score_

0 : -0.000362130561095
1 : -0.000205921770629
2 : -0.000258223690464
3 : -0.000300647521563
4 : -0.00018345738362
5 : -0.000194744154298
6 : -0.000280869673447
7 : -0.000204093226766
8 : -0.000352471754087
9 : -0.000322927543143
10 : -0.000276430826343


In [15]:
s = 0
for i in xrange(11):
    s += -1 * clf[i].best_score_
import math
print math.sqrt(s/11.0)

0.0163538099688


fit test data

In [16]:
Test = TemperatureData('test')
tstempers = Test.temperWVecMean()

In [21]:
tstempers_place = dict()
tslearnig_id = dict()
for i in xrange(11):
    ls = list(Test.TemperPlaceId[i])
    tstempers_place[i] = tstempers[ls]
    tslearnig_id[i] = ls

In [22]:
result = np.zeros((len(Test.temperVec()),))
for i in xrange(11):
    print i
    result[tslearnig_id[i]] = clf[i].predict(tstempers_place[i])

0
1
2
3
4
5
6
7
8
9
10


In [23]:
np.savetxt("../result/sub.dat", result, fmt='%.10f')

In [17]:
[len(Test.TemperPlaceId[i]) for i in xrange(11)]

[164, 164, 164, 164, 164, 164, 164, 163, 163, 163, 163]

In [25]:
tstempers_place = dict()
tslearnig_id = dict()
tsdrop_list = list()
for i in xrange(11):
    drop_flag = False
    for edge in edges:
        if edge in Test.TemperPlaceId[i]:
            a = list(Test.TemperPlaceId[i])
            index = a.index([edge])
            tsdrop_list.append([i,index])
            ls = list(Test.TemperPlaceId[i])
            ls.pop(index)
            tstempers_place[i] = tstempers[ls]
            tslearnig_id[i] = ls
            drop_flag = True
            break
    if not drop_flag:
        ls = list(Test.TemperPlaceId[i])
        tstempers_place[i] = tstempers[ls]
        tslearnig_id[i] = ls

In [26]:
[len(tstempers_place[i]) for i in xrange(11)]

[164, 163, 164, 164, 163, 164, 163, 162, 163, 162, 163]

In [27]:
[len(tslearnig_id[i]) for i in xrange(11)]

[164, 163, 164, 164, 163, 164, 163, 162, 163, 162, 163]

In [28]:
result = np.zeros((len(Test.temperVec()),))
for i in xrange(11):
    print i
    result[tslearnig_id[i]] = clf[i].predict(tstempers_place[i])

0
1
2
3
4
5
6
7
8
9
10


In [29]:
result[360]

0.090861053295334199

In [30]:
tsp_result = np.loadtxt("../result/eachPlace_0.03083.dat")

In [31]:
tsp_result[359]

-0.12392710630000001

In [32]:
for edge in edges:
    result[edge] = tsp_result[edge] 

In [33]:
np.savetxt("../result/sub.dat", result, fmt='%.10f')

In [36]:
result[4]

0.25494819229527271

In [30]:
edges

[359, 719, 1079, 1439, 1799]

## making preprocessing (temperWVec) to change before-after-data

In [263]:
yearnum = 5
placenum = len(Train.TemperPlaceId)
datasize_year = 360
stk = np.array([])
for n in xrange(yearnum):
    start = datasize_year*n
    end = datasize_year*(n+1) -1
    tempers_year = Train.Temperature.loc[start:end,  ['place%d' % i for i in xrange(placenum)]].values
    for i in xrange(datasize_year-1):
        vec = np.r_[tempers_year[i],tempers_year[i+1]]
        stk = np.append(stk, vec)
    last = np.append(tempers_year[datasize_year-1],np.zeros((placenum)))
    stk = np.append(stk, last)
result = stk.reshape(datasize_year*5, placenum*2)

In [274]:
result[359], result[719], result[1079],result[1439], result[1799]

(array([-0.183283, -0.183283, -0.211585, -0.243032, -0.132969, -0.192717,
        -0.085799, -0.098378, -0.123535, -0.170705, -0.189573,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ]),
 array([-0.199007, -0.195862, -0.2619  , -0.25561 , -0.158126, -0.224164,
        -0.117246, -0.098378, -0.110956, -0.224164, -0.236742,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ]),
 array([-0.16756 , -0.208441, -0.21473 , -0.202151, -0.16756 , -0.170705,
        -0.117246, -0.129824, -0.151837, -0.173849, -0.217874,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ]),
 array([-0.148692, -0.161271, -0.173849, -0.180139, -0.145547, -0.151837,
        -0.136113, -0.132969, -0.129824, -0.151837, -0.180139,  0.      ,
         0.      ,  0.  